In [1]:
# importing packages

import os
import glob
import numpy as np
import pandas as pd
from datetime import datetime

from openai import AzureOpenAI

from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

import chromadb
from chromadb.utils import embedding_functions

import tiktoken
from tqdm import tqdm
from glob import glob
from dotenv import load_dotenv
import xml.etree.ElementTree as ET  

In [2]:
# loading env variable
load_dotenv()

True

In [3]:
## GPT-3.5
client = AzureOpenAI(
  azure_endpoint = "https://cog-cqz45retsstie.openai.azure.com/", 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

In [15]:
# prompt
header = """Definition: You are an experient software engineer that works with code translation. Your knowledge is based o the R and Python languages. You can convert any type of code from R to Python.\n"""

context = """Context:
You will be given a R code and you will perform as follows:
1. You are going to translate the R code to Python;
2. If there are any dependencies on R, you will translate to their Python's corresnpondents;
3. You MUST add comments on each piece of code translated, explaining its behaviour. You MUST be very descriptive;
4. You will translate ONLY the code given, do not create new one.
5. You will create a requirements file with the dependencies that must be installed in Python.\n"""

action = """
Action:
Given the following R code, convert it into Python, and then, return a JSON with `PyCode` as key and the correspondent Python code translated as value, and a second key `Reqs` with the requirements to install the dependencies as value:

```{code}```

RETURN ONLY THE JSON OBJECT. NOTHING ELSE.
The result JSON is:\n
"""

code = """
library(“copula”, lib=installLocation)
library(“ggplot2”, lib=installLocation)
library(“scatterplot3d”, lib=installLocation)
library(“grid”, lib=installLocation)
library(“ abelling”, lib=installLocation)
library(“VineCopula”, lib=installLocation)
library(“fitdistrplus”, lib=installLocation)

# Generate a bivariate normal copula with rho = 0.7
normal <- normalCopula(param = 0.7, dim = 2)
# Generate a bivariate t-copula with rho = 0.8 and df = 2
stc <- tCopula(param = 0.8, dim = 2, df = 2)

# Build a Frank, a Gumbel and a Clayton copula 
frank <- frankCopula(dim = 2, param = 8) 
gumbel <- gumbelCopula(dim = 3, param = 5.6) 
clayton <- claytonCopula(dim = 4, param = 19)
"""

template = header + context + action

prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(code=code)

message_text = [{"role":"system",
                 "content":prompt},]

In [16]:
completion = client.chat.completions.create(
  model="chat", # model = "deployment_name"
  messages = message_text,
  temperature=0.1,
  max_tokens=2048,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

print(completion.choices[0].message.content)

code_translated = eval(completion.choices[0].message.content)

{
    "PyCode": "import copula\nimport ggplot2\nimport scatterplot3d\nimport grid\nimport abelling\nimport VineCopula\nimport fitdistrplus\n\n# Generate a bivariate normal copula with rho = 0.7\nnormal = copula.normalCopula(param = 0.7, dim = 2)\n# Generate a bivariate t-copula with rho = 0.8 and df = 2\nstc = copula.tCopula(param = 0.8, dim = 2, df = 2)\n\n# Build a Frank, a Gumbel and a Clayton copula \nfrank = copula.frankCopula(dim = 2, param = 8) \ngumbel = copula.gumbelCopula(dim = 3, param = 5.6) \nclayton = copula.claytonCopula(dim = 4, param = 19)", 
    "Reqs": "copula\nggplot2\nscatterplot3d\ngrid\nabelling\nVineCopula\nfitdistrplus"
}


In [10]:
code_translated['PyCode']

'import copula\nimport ggplot2\nimport scatterplot3d\nimport grid\nimport abelling\nimport VineCopula\nimport fitdistrplus\n\n# Generate a bivariate normal copula with rho = 0.7\nnormal = copula.normalCopula(param = 0.7, dim = 2)\n# Generate a bivariate t-copula with rho = 0.8 and df = 2\nstc = copula.tCopula(param = 0.8, dim = 2, df = 2)\n\n# Build a Frank, a Gumbel and a Clayton copula \nfrank = copula.frankCopula(dim = 2, param = 8) \ngumbel = copula.gumbelCopula(dim = 3, param = 5.6) \nclayton = copula.claytonCopula(dim = 4, param = 19)'

In [17]:
def save_python_script(dict_code, filename):
  """
  Saves the provided Python code string as a Python script file.

  Args:
      dict_code (Dict): The Python Dictionary Object with the code string and the requirements to be saved.
      filename (str): The desired filename for the Python script (should end with '.py').
  """

  try:
    with open(filename, 'w') as f:
      f.write(dict_code['PyCode'])
    with open('reqs.txt', 'w') as f:
      f.write(dict_code['Reqs'])
      
    print(f"Python script saved successfully: {filename}")
  except OSError as e:
    print(f"Error saving Python script: {e}")

In [18]:
save_python_script(code_translated, 'Python_code.py')

Python script saved successfully: Python_code.py
